In [ ]:
import boto3

# Especifica el perfil explícitamente (si usas un perfil diferente)
session = boto3.Session(profile_name="Pragma-Bedrock")  # Cambia "default" si usas otro perfil
credentials = session.get_credentials().get_frozen_credentials()

print(f"AWS_ACCESS_KEY_ID: {credentials.access_key}")
print(f"AWS_SECRET_ACCESS_KEY: {'*' * len(credentials.secret_key)} (ocultado por seguridad)")
if credentials.token:
    print(f"AWS_SESSION_TOKEN: {'*' * len(credentials.token)} (ocultado por seguridad)")
else:
    print("No se encontró AWS_SESSION_TOKEN.")

# Proceso de Recopilación y Organización de Documentos PDF

En esta etapa del flujo de trabajo, se llevará a cabo la recopilación y organización de documentos PDF de los usuarios. A continuación, se detallan los pasos a seguir:

1. **Definición de la Función `docs_for_clients`**:
    - Se definirá una función que recorrerá un directorio base (`ruta_base`) y recopilará todos los archivos PDF de los subdirectorios. 
    - Los documentos se almacenarán en un diccionario donde las claves serán los nombres de los usuarios (nombres de las carpetas) y los valores serán las rutas de los archivos PDF.

2. **Iteración sobre Directorios y Archivos**:
    - La función iterará sobre todos los directorios y archivos dentro de la ruta base utilizando `os.walk`.
    - Se filtrarán los archivos que terminen en `.pdf` y se generarán las rutas completas de estos archivos.

3. **Almacenamiento de Documentos en un Diccionario**:
    - Si se encuentran archivos PDF, se almacenarán en un diccionario con el nombre del usuario (nombre de la carpeta) como clave y las rutas de los archivos PDF como valores.

4. **Ejemplo de Uso**:
    - Se definirá una ruta base (`ruta_base`) y se llamará a la función `docs_for_clients` para obtener el diccionario de documentos de los usuarios.
    - Se mostrarán los resultados, imprimiendo el nombre del usuario y las rutas de sus archivos PDF.

Este proceso permitirá tener una estructura organizada de los documentos PDF de los usuarios, facilitando su posterior procesamiento y análisis.

In [ ]:
import os

def docs_for_clients(ruta_base):
    documentos_usuarios = {}
    
    # Iterar sobre todos los directorios y archivos dentro de la ruta base
    for ruta_directorio, subdirectorios, archivos in os.walk(ruta_base):
        # Obtener el nombre de la carpeta (nombre del usuario)
        nombre_usuario = os.path.basename(ruta_directorio)
        
        # Filtrar los archivos que terminan en .pdf
        pdfs = [archivo for archivo in archivos if archivo.endswith('.pdf')]
        
        # Si se encuentran archivos PDF, almacenarlos con el nombre del usuario
        if pdfs:
            # Generar las rutas completas de los archivos PDF
            rutas_pdfs = [os.path.join(ruta_directorio, pdf) for pdf in pdfs]
            # Asociar el nombre del usuario (nombre de la carpeta) con los PDFs
            documentos_usuarios[nombre_usuario] = rutas_pdfs
    
    return documentos_usuarios

# Ejemplo de uso
ruta_base = "documentos/"
data_usuarios = docs_for_clients(ruta_base)

# Mostrar los resultados
for usuario, pdfs in data_usuarios.items():
    print(f"Usuario: {usuario}")
    for pdf in pdfs:
        print(f" - {pdf}")


In [24]:
import boto3
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    # Initialize Textract client
    textract = session.client('textract')

    # Open the PDF document
    pdf_document = fitz.open(file_path)
    extracted_text = ""

    # Loop through each page in the PDF
    for page_number in range(len(pdf_document)):
        # Extract the page as an image (pixmap)
        page = pdf_document[page_number]
        pix = page.get_pixmap()

        # Save the image to a temporary file
        image_bytes = pix.tobytes()

        # Call Textract to extract text from the image
        response = textract.detect_document_text(Document={'Bytes': image_bytes})

        # Extract text from the response
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                extracted_text += item["Text"] + "\n"


    pdf_document.close()  # Close the PDF document
    return extracted_text


# Análisis de Texto Extraído de Documentos PDF

En esta etapa del flujo de trabajo, se llevará a cabo la adición del texto extraído de los documentos PDF al diccionario de datos de los usuarios. A continuación, se detallan los pasos a seguir:

1. **Definición de la Función `agregar_texto_a_documentos`**:
    - Se define una función que itera sobre cada usuario en el diccionario `datos_usuario`.
    - Para cada usuario, se itera sobre cada archivo PDF asociado.

2. **Extracción de Texto de los PDFs**:
    - Para cada PDF, se utiliza la función `extract_text_from_pdf` para extraer el texto del documento.
    - El texto extraído se almacena en una nueva estructura de datos que reemplaza la ruta del archivo PDF con un diccionario que contiene tanto la ruta como el texto extraído.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario` se actualiza para incluir el texto extraído de cada PDF.
    - La función devuelve el diccionario actualizado con la nueva estructura de datos.

4. **Ejecución de la Función**:
    - Se ejecuta la función `agregar_texto_a_documentos` pasando el diccionario `data_usuarios` como argumento.
    - El resultado es un nuevo diccionario `datos_usuario_con_texto` que contiene tanto las rutas de los archivos PDF como el texto extraído de cada uno.

Este proceso es fundamental para preparar los datos de los documentos PDF para su posterior análisis y generación de informes financieros.

In [ ]:

def agregar_texto_a_documentos(datos_usuario):
    # Iterar sobre cada usuario en el diccionario
    for usuario, pdfs in datos_usuario.items():
        # Iterar sobre cada PDF del usuario
        for i, ruta_pdf in enumerate(pdfs):
            # Extraer el texto del PDF usando tu método extract_text_from_pdf
            texto_pdf = extract_text_from_pdf(ruta_pdf)
            
            # Reemplazar el string de la ruta con un diccionario que contenga la ruta y el texto
            datos_usuario[usuario][i] = {
                'ruta': ruta_pdf,
                'texto': texto_pdf
            }

    return datos_usuario


# Ejecutar la función para agregar el texto extraído
datos_usuario_con_texto = agregar_texto_a_documentos(data_usuarios)
datos_usuario_con_texto


### Análisis de Texto Extraído y Generación de Informe de Crédito

En esta etapa del flujo de trabajo, se llevará a cabo el análisis del texto extraído de los documentos PDF y la generación de un informe de crédito detallado. A continuación, se detallan los pasos a seguir:

1. **Iteración sobre Usuarios y Documentos**:
    - Se iterará sobre cada usuario en el diccionario `datos_usuario_con_texto`.
    - Para cada usuario, se iterará sobre cada archivo PDF asociado.

2. **Análisis del Texto Extraído**:
    - Para cada PDF, se utilizará la función `analizeTitan` para analizar el texto extraído del documento.
    - El resultado del análisis se almacenará en una nueva clave dentro del diccionario del PDF.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario_con_texto` se actualizará para incluir el resultado del análisis de cada PDF.
    - Se generará un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada.

4. **Generación del Informe de Crédito**:
    - Se configurará un mensaje del sistema que especifica las instrucciones para el modelo de IA encargado de generar el informe de crédito.
    - El modelo de IA producirá un informe de crédito basado en el análisis de la información financiera del usuario.

5. **Visualización del Informe**:
    - Se utilizará la biblioteca `IPython.display` para mostrar el informe de crédito en formato Markdown, facilitando su lectura y comprensión.

Este proceso es fundamental para evaluar la viabilidad crediticia de los usuarios, proporcionando un análisis detallado de sus datos financieros y generando informes que pueden ser utilizados para la toma de decisiones.

In [160]:
file_path = 'prueba.pdf'
text = extract_text_from_pdf(file_path)

# Mensaje del sistema con instrucciones para el modelo Titan Text G1
system_message = """
Eres una IA especializada en la extracción de datos financieros desde documentos bancarios en formato PDF o imagen, estos documentos pueden ser extractos
de productos bancarios como tarjetas de credito, cuentas de ahorro, cuentas corrientes, cdts, hipotecas. Tambien peuden consistir en comprobantes de 
contabilidad, soportes de ingresos, tirillas de pago, cartas laborales con salario, etc. 
Tu tarea es analizar el contenido del documento, identificar las transacciones, fechas, montos, debitos, creditos, movimientos negativos y positivos, 
pagos y otros detalles financieros relevantes, y estructurar estos datos en un formato JSON organizado. 
Eres capaz de procesar documentos con diferentes formatos y estructuras, sin depender de plantillas predefinidas. 
La salida debe contener los siguientes campos clave: número de contrato, pagos, transacciones, cargos adicionales, fechas, montos y saldos pendientes. 
Si hay información adicional relevante (por ejemplo, tasas de interés, detalles del crédito), también debes incluirla en el JSON.
"""

# Análisis de Texto Extraído y Generación de Informe de Crédito

En esta etapa del flujo de trabajo, se llevará a cabo el análisis del texto extraído de los documentos PDF y la generación de un informe de crédito detallado. A continuación, se detallan los pasos a seguir:

1. **Iteración sobre Usuarios y Documentos**:
    - Se iterará sobre cada usuario en el diccionario `datos_usuario_con_texto`.
    - Para cada usuario, se iterará sobre cada archivo PDF asociado.

2. **Análisis del Texto Extraído**:
    - Para cada PDF, se utilizará la función `analizeTitan` para analizar el texto extraído del documento.
    - El resultado del análisis se almacenará en una nueva clave dentro del diccionario del PDF.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario_con_texto` se actualizará para incluir el resultado del análisis de cada PDF.
    - Se generará un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada.

4. **Generación del Informe de Crédito**:
    - Se configurará un mensaje del sistema que especifica las instrucciones para el modelo de IA encargado de generar el informe de crédito.
    - El modelo de IA producirá un informe de crédito basado en el análisis de la información financiera del usuario.

5. **Visualización del Informe**:
    - Se utilizará la biblioteca `IPython.display` para mostrar el informe de crédito en formato Markdown, facilitando su lectura y comprensión.

Este proceso es fundamental para evaluar la viabilidad crediticia de los usuarios, proporcionando un análisis detallado de sus datos financieros y generando informes que pueden ser utilizados para la toma de decisiones.

In [161]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

def analizeTitan(system_message,text):

    body = json.dumps({
        "inputText": f"Instructions: {system_message}. Human: Este es el texto extraido: {text}. AI:",
        "textGenerationConfig": {
            "maxTokenCount": 3000,
            "temperature": 0.2,
            "topP": 1
        }
    })

    modelId = 'amazon.titan-text-premier-v1:0'
    accept = 'application/json'
    contentType = 'application/json'

    response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

    response_body = json.loads(response.get('body').read())

    # text
    data = json.loads(response_body['results'][0]['outputText'])
    return data

# Análisis de Texto Extraído y Generación de Informe de Crédito

En esta etapa del flujo de trabajo, se llevará a cabo el análisis del texto extraído de los documentos PDF y la generación de un informe de crédito detallado. A continuación, se detallan los pasos a seguir:

1. **Iteración sobre Usuarios y Documentos**:
    - Se iterará sobre cada usuario en el diccionario `datos_usuario_con_texto`.
    - Para cada usuario, se iterará sobre cada archivo PDF asociado.

2. **Análisis del Texto Extraído**:
    - Para cada PDF, se utilizará la función `analizeTitan` para analizar el texto extraído del documento.
    - El resultado del análisis se almacenará en una nueva clave dentro del diccionario del PDF.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario_con_texto` se actualizará para incluir el resultado del análisis de cada PDF.
    - Se generará un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada.

4. **Generación del Informe de Crédito**:
    - Se configurará un mensaje del sistema que especifica las instrucciones para el modelo de IA encargado de generar el informe de crédito.
    - El modelo de IA producirá un informe de crédito basado en el análisis de la información financiera del usuario.

5. **Visualización del Informe**:
    - Se utilizará la biblioteca `IPython.display` para mostrar el informe de crédito en formato Markdown, facilitando su lectura y comprensión.

Este proceso es fundamental para evaluar la viabilidad crediticia de los usuarios, proporcionando un análisis detallado de sus datos financieros y generando informes que pueden ser utilizados para la toma de decisiones.

In [ ]:
for usuario, pdfs in datos_usuario_con_texto.items():
    # Iterar sobre cada PDF asociado a ese usuario
    for pdf in pdfs:
        # Obtener el texto del PDF
        texto_pdf = pdf['texto']
        
        # Llamar al método analize_text y almacenar el resultado en una nueva clave
        resultado_analisis = analizeTitan(system_message,texto_pdf)
        pdf['resultado_analisis'] = resultado_analisis  # Agregar como nuevo key:value

datos_usuario_con_texto

# Generación de Informe de Crédito

En esta etapa del flujo de trabajo, se llevará a cabo la generación de un informe de crédito detallado basado en el análisis del texto extraído de los documentos PDF. A continuación, se detallan los pasos a seguir:

1. **Iteración sobre Usuarios y Documentos**:
    - Se iterará sobre cada usuario en el diccionario `datos_usuario_con_texto`.
    - Para cada usuario, se iterará sobre cada archivo PDF asociado.

2. **Análisis del Texto Extraído**:
    - Para cada PDF, se utilizará la función `analizeTitan` para analizar el texto extraído del documento.
    - El resultado del análisis se almacenará en una nueva clave dentro del diccionario del PDF.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario_con_texto` se actualizará para incluir el resultado del análisis de cada PDF.
    - Se generará un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada.

4. **Generación del Informe de Crédito**:
    - Se configurará un mensaje del sistema que especifica las instrucciones para el modelo de IA encargado de generar el informe de crédito.
    - El modelo de IA producirá un informe de crédito basado en el análisis de la información financiera del usuario.

5. **Visualización del Informe**:
    - Se utilizará la biblioteca `IPython.display` para mostrar el informe de crédito en formato Markdown, facilitando su lectura y comprensión.

Este proceso es fundamental para evaluar la viabilidad crediticia de los usuarios, proporcionando un análisis detallado de sus datos financieros y generando informes que pueden ser utilizados para la toma de decisiones.

In [ ]:
from IPython.display import Markdown, display

body2 = json.dumps({
    "inputText": f"""Instructions: Eres un analista de contabilidad, recibes informes en formato diccionario y los devuelves en formato markdown para que sean faciles
    de ver y entender para el usuario final. 
    Human: Este es el usuario a evaluar: {list(datos_usuario_con_texto.keys())[0]} y este es su diccionario: {datos_usuario_con_texto}. AI:""",
    "textGenerationConfig": {
        "maxTokenCount": 3000,
        "temperature": 0.2,
        "topP": 1
    }
})

modelId = 'amazon.titan-text-premier-v1:0'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model(body=body2, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

# text

display(Markdown(response_body['results'][0]['outputText']))

# Generación de Informe de Crédito

En esta etapa del flujo de trabajo, se llevará a cabo la generación de un informe de crédito detallado basado en el análisis del texto extraído de los documentos PDF. A continuación, se detallan los pasos a seguir:

1. **Iteración sobre Usuarios y Documentos**:
    - Se iterará sobre cada usuario en el diccionario `datos_usuario_con_texto`.
    - Para cada usuario, se iterará sobre cada archivo PDF asociado.

2. **Análisis del Texto Extraído**:
    - Para cada PDF, se utilizará la función `analizeTitan` para analizar el texto extraído del documento.
    - El resultado del análisis se almacenará en una nueva clave dentro del diccionario del PDF.

3. **Actualización del Diccionario de Datos**:
    - El diccionario `datos_usuario_con_texto` se actualizará para incluir el resultado del análisis de cada PDF.
    - Se generará un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada.

4. **Generación del Informe de Crédito**:
    - Se configurará un mensaje del sistema que especifica las instrucciones para el modelo de IA encargado de generar el informe de crédito.
    - El modelo de IA producirá un informe de crédito basado en el análisis de la información financiera del usuario.

5. **Visualización del Informe**:
    - Se utilizará la biblioteca `IPython.display` para mostrar el informe de crédito en formato Markdown, facilitando su lectura y comprensión.

Este proceso es fundamental para evaluar la viabilidad crediticia de los usuarios, proporcionando un análisis detallado de sus datos financieros y generando informes que pueden ser utilizados para la toma de decisiones. Además permite evaluar el comportamiento del modelo, verificar posibles alucinaciones o confusiones en el orden de los datos

In [ ]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime')

primera_clave = list(datos_usuario_con_texto.keys())[0]
datos = {}
datos[primera_clave] = {}
for i in range(len(datos_usuario_con_texto[primera_clave])):
    datos[primera_clave]["analisis documento " + str(i+1)] = datos_usuario_con_texto[primera_clave][i]['resultado_analisis']


body = json.dumps({
    "inputText": f"""Instructions: Eres un analista de credito, recibiras un json con los datos financieros de un usuario y tu los analizaras y 
    generaras un informe de credito del usuario, el objetivo es considerar la viabilidad del usuario para recibir productos crediticios, Genera un analisis
    conciso y preciso, priorizando las cifras de saldos a favor y en contra asi como posibles capacidades de endeudamiento y demas datos que consideres de 
    valor, Debes dar tu recomendacion al final y las razones de la misma.. 
    Human: Este es el nombre del usuario: {primera_clave} y este es su json: {datos[primera_clave]}. AI:""",
    "textGenerationConfig": {
        "maxTokenCount": 1500,
        "temperature": 0.2,
        "topP": 1
    }
})

modelId = 'amazon.titan-text-premier-v1:0'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

# text
display(Markdown(response_body['results'][0]['outputText']))

## Flujo de Trabajo

1. **Recopilación de Documentos**:
   - Se define una función `docs_for_clients(ruta_base)` que recorre un directorio base (`ruta_base`) y recopila todos los archivos PDF de subdirectorios. Los documentos se almacenan en un diccionario donde las claves son los nombres de los usuarios y los valores son las rutas de los archivos PDF.

2. **Extracción de Texto de Documentos PDF**:
   - Se utiliza `extract_text_from_pdf(file_path)` para extraer texto de cada archivo PDF. Se emplea la biblioteca **PyMuPDF** para abrir el PDF y se envían las páginas a **Amazon Textract** para la extracción del texto.

3. **Organización de Datos**:
   - Se define `agregar_texto_a_documentos(datos_usuario)` para reemplazar las rutas de los archivos PDF en el diccionario con un diccionario que contenga la ruta y el texto extraído de cada PDF.

4. **Análisis de Texto Extraído**:
   - Se configura un mensaje del sistema (`system_message`) que especifica las instrucciones para el modelo de IA (Amazon Titan Text G1) encargado de analizar el texto extraído y organizarlo en formato JSON con campos clave como transacciones, montos, fechas, etc.
   - La función `analizeTitan(system_message, text)` invoca el modelo de IA, pasando el texto extraído y las instrucciones. La respuesta se convierte en un objeto JSON estructurado.

5. **Generación de Informe de Crédito**:
   - Se itera sobre los usuarios y sus documentos para analizar el JSON generado por el modelo de IA y agregar el resultado del análisis a la estructura de datos original.
   - Se genera un nuevo cuerpo de solicitud para el modelo de IA, pasando el JSON de análisis como entrada, para que produzca un informe de crédito.

6. **Salida de Resultados**:
   - Se imprimen o guardan los resultados finales que contienen el informe de crédito de cada usuario basado en el análisis de su información financiera.

### Arquitectura de Recursos y Herramientas

```plaintext
                                ┌────────────────────────────────────────┐
                                │               Usuario                 │
                                │ (Sube documentos PDF a S3)           │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │             AWS S3                    │
                                │  (Almacenamiento de documentos PDF)    │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │             AWS Lambda                │
                                │   (Función que inicia el flujo)       │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │            Amazon Textract            │
                                │    (Extracción de texto de PDF)       │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │           AWS Lambda                  │
                                │   (Procesa texto extraído y lo envía) │
                                │     a Amazon Bedrock para análisis    │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │          Amazon Bedrock               │
                                │    (Modelo Titan Text G1 para         │
                                │    análisis de datos financieros)      │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │            AWS Lambda                 │
                                │  (Generación de informes de crédito)  │
                                └────────────────────────────────────────┘
                                                    │
                                                    ▼
                                ┌────────────────────────────────────────┐
                                │               AWS S3                  │
                                │    (Almacena informes generados)      │
                                └────────────────────────────────────────┘
```

---

### Descripción de Componentes:

1. **Usuario**:
   - Interactúa con la aplicación subiendo documentos PDF a un bucket de Amazon S3.

2. **AWS S3**:
   - Almacena los documentos PDF subidos por los usuarios.

3. **AWS Lambda**:
   - **Primera función**: Se activa al subir un nuevo PDF y gestiona el flujo de trabajo.
   - **Segunda función**: Procesa el texto extraído de Amazon Textract y lo envía a Amazon Bedrock para análisis.
   - **Tercera función**: Genera informes de crédito a partir de los datos analizados.

4. **Amazon Textract**:
   - Extrae texto y datos de los documentos PDF almacenados en S3.

5. **Amazon Bedrock**:
   - Utiliza el modelo Titan Text G1 para analizar el texto extraído y estructurar los datos financieros en formato JSON.

6. **AWS S3 (Informes)**:
   - Almacena los informes de crédito generados para su posterior acceso y revisión.

---

